In [1]:
import numpy as np
import tensorflow as tf
from keras import *
from keras.layers import *
from keras import backend as K
from keras.utils import np_utils 
from keras.utils import plot_model
from keras.utils import to_categorical
from keras.models import *
from keras.optimizers import SGD
import cv2
import glob
import pydot
import graphviz
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

/home/shyam/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# VGG16

## VGG16 Net implemented from scratch with Keras
## based on the paper: Very Deep Convolutional Networks for Large-Scale Image Recognition

## https://arxiv.org/abs/1409.1556


### The data is from a dataset of flower images, with 10 different classes or types of flowers. We will resize the images to be 224x224x3

In [2]:
data = pd.read_csv("data/flower_images/flower_labels.csv")
data["file"] = "data/flower_images/" + data["file"]
X = np.array(data.iloc(1)[0])
y = np.array(data.iloc(1)[1])

In [3]:
def cross_val_split_set(X,portion,y=None):
    '''
    use:
        X = iris.data
        y = iris.target
        X_train, X_test, y_train, y_test = split_set(X,0.1,y) for 10% used as test set
    '''
    X = np.array(X)
    y = np.array(y)
    size = int(X.shape[0]*portion)
    indexlist = np.arange(X.shape[0])
    testinds = np.random.choice(indexlist, size, replace=False)
    traininds = np.array([x for x in range(X.shape[0]) if x not in testinds])  
    if np.all(y == None):
        return X[traininds],X[testinds]
    else:
        return X[traininds],X[testinds],y[traininds],y[testinds]

X_train, X_test, y_train, y_test = cross_val_split_set(X,0.1,y)

In [4]:
y_train = y_train.reshape(y_train.shape[0],1)
y_test = y_test.reshape(y_test.shape[0],1)

In [5]:
def preprocess(path,length,width):
    img = cv2.imread(path)
    img = np.asfarray(cv2.resize(img,(length, width)))
    return img/255.0

def read_bulk(paths):
    result = []
    for i in paths:
        result.append(preprocess(i,224,224))
    return np.array(result)

result = read_bulk(X_train)

In [6]:
class VGG16():
    """
    VGG16 net implemented with keras
    introduced in the paper "Very Deep Convolutional Networks for Large-Scale Image Recognition"
    https://arxiv.org/abs/1409.1556
    """
    def __init__(self,X,y,weights="None"):
        
        self.X = X
        self.y = to_categorical(y)
                
        self.weights = weights
        self.model = None
        
        if self.weights == "None":
            self.initialize()
        else:
            self.model = load_model(weights)
        
    def initialize(self):
        
        K.clear_session()
        
        n_outputs = self.y.shape[1]
        height = self.X.shape[1]
        width = self.X.shape[2]
        
        inp = Input(shape=(height,width,3))
        conv1 = Conv2D(64,kernel_size=3,border_mode='same',activation="relu")(inp)
        conv2 = Conv2D(64,kernel_size=3,border_mode='same',activation='relu')(conv1)
        max1 = MaxPool2D(2,border_mode='same')(conv2)
        dropout1 = Dropout(0.5)(max1)
        conv3 = Conv2D(128,kernel_size=3,border_mode='same',activation='relu')(dropout1)
        conv4 = Conv2D(128,kernel_size=3,border_mode='same',activation='relu')(conv3)
        max2 = MaxPool2D(2,border_mode='same')(conv4)
        dropout2 = Dropout(0.5)(max2)
        conv5 = Conv2D(256,kernel_size=3,border_mode='same',activation='relu')(dropout2)
        conv6 = Conv2D(256,kernel_size=3,border_mode='same',activation='relu')(conv5)
        conv7 = Conv2D(256,kernel_size=3,border_mode='same',activation='relu')(conv6)
        max3 = MaxPool2D(2,border_mode='same')(conv7)
        dropout3 = Dropout(0.5)(max3)
        conv8 = Conv2D(512,kernel_size=3,border_mode='same',activation='relu')(dropout3)
        conv9 = Conv2D(512,kernel_size=3,border_mode='same',activation='relu')(conv8)
        conv10 = Conv2D(512,kernel_size=3,border_mode='same',activation='relu')(conv9)
        max4 = MaxPool2D(2,border_mode='same')(conv10)
        dropout4 = Dropout(0.5)(max4)
        conv11 = Conv2D(512,kernel_size=3,border_mode='same',activation='relu')(dropout4)
        conv12 = Conv2D(512,kernel_size=3,border_mode='same',activation='relu')(conv11)
        conv13 = Conv2D(512,kernel_size=3,border_mode='same',activation='relu')(conv12)
        max5 = MaxPool2D(2,border_mode='same')(conv13)
        dropout5 = Dropout(0.5)(max5)
        flatten = Flatten()(dropout5)
        dense1 = Dense(4096,activation='relu')(flatten)
        dropout6 = Dropout(0.5)(dense1)
        dense2 = Dense(4096,activation='relu')(dropout6)
        dropout7 = Dropout(0.5)(dense2)
        dense3 = Dense(n_outputs,activation='relu')(dropout7)
        dropout8 = Dropout(0.5)(dense3)
        softmax = Softmax(n_outputs)(dropout8)
        
        model = Model(inputs=inp,outputs=softmax)
        model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])
        self.model = model
        print(self.model.summary())
        
    def save_picture(self,filename):
        plot_model(self.model, to_file=filename)
        
    def train(self,epochs):

        self.model.fit(self.X, self.y ,validation_split=0.1, epochs=epochs,verbose=1)
        self.model.save('saved_models/VGG16.h5')
        loss, acc = self.model.evaluate(self.X, self.y, verbose=0)
        print('Train Accuracy: %f' % (acc*100))
        
    def predict(self,X):
        
        if len(X.shape) == 3:
            X = X.reshape(1,X.shape[0],X.shape[1],X.shape[2])
        predictions = self.model.predict(X)
        return np.argmax(predictions)
        

In [7]:
vgg16 = VGG16(result,y_train)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
__________

In [35]:
vgg16.save_picture("../model_images/VGG16.png")

In [ ]:
vgg16.train(2)

Train on 170 samples, validate on 19 samples
Epoch 1/2
128/170 [=====================>........] - ETA: 7:30 - loss: 3.5123 - acc: 0.0703 